In [14]:
import numpy as np
import pandas as pd
import plotly as plt
import scipy
import math
from datetime import datetime

In [15]:
df_matches = pd.read_csv("../data/cleaned/tennis_matches_cleaned.csv")
df_male_players = pd.read_csv("../data/cleaned/male_players_cleaned.csv")
df_female_players = pd.read_csv("../data/cleaned/female_players_cleaned.csv")

# Data integration

Create a dataframe with all the male matches and another one for the female matches

In [16]:
df_male_players_tmp = pd.DataFrame()
df_male_players_tmp['player_name'] = df_male_players['name'] + ' ' + df_male_players['surname']
df_male_winners = df_male_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_male_losers = df_male_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_male_matches = pd.concat([df_male_winners, df_male_losers])

In [17]:
df_male_matches

,player_name,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,loser_rank,tourney_spectators,tourney_revenue,winner_name
643,Alexander Zverev,352,2019-580,Australian Open,Hard,128,G,2019-01-14,200,100644,...,37.0,12.0,13.0,6.0,12.0,4,155,15583.0,2781000.18,NaN
643,Alexander Zverev,363,2019-580,Australian Open,Hard,128,G,2019-01-14,116,100644,...,34.0,8.0,13.0,2.0,8.0,4,67,15583.0,2781000.18,NaN
643,Alexander Zverev,364,2019-580,Australian Open,Hard,128,G,2019-01-14,172,100644,...,86.0,29.0,26.0,12.0,17.0,4,36,15583.0,2781000.18,NaN
643,Alexander Zverev,1145,2019-M004,Acapulco,Hard,32,A,2019-02-25,298,100644,...,29.0,8.0,10.0,2.0,4.0,3,64,3398.0,655207.08,NaN
643,Alexander Zverev,1149,2019-M004,Acapulco,Hard,32,A,2019-02-25,294,100644,...,28.0,11.0,10.0,0.0,2.0,3,26,3398.0,655207.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54476,Michele Vianello,94508,2021-9357,Forli CH,Clay,32,C,2021-06-14,243,104563,...,21.0,9.0,9.0,2.0,6.0,433,-1,3031.0,566031.89,Luca Vanni
54476,Michele Vianello,95338,2021-2120,Cordenons CH,Clay,32,C,2021-08-02,250,207608,...,28.0,5.0,10.0,2.0,7.0,291,-1,2876.0,543907.53,Timofey Skatov
54485,Ben Shelton,95003,2021-7316,Cary CH,Hard,32,C,2021-07-19,254,206499,...,24.0,12.0,9.0,3.0,5.0,413,1835,3074.0,583894.56,Aleksandar Kovacevic
54504,William Jansen,64393,2021-540,Wimbledon,Grass,128,G,2021-06-28,138,207760,...,28.0,17.0,13.0,8.0,16.0,454,-1,14566.0,5002793.97,Anton Matusevich


df_male_matches.describe()

In [18]:
df_female_players_tmp = pd.DataFrame()
df_female_players_tmp['player_name'] = df_female_players['name'] + ' ' + df_female_players['surname']
df_female_winners = df_female_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_female_losers = df_female_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_female_matches = pd.concat([df_female_winners, df_female_losers])

In [19]:
df_male_matches_reduced = df_male_losers[['player_name', 'winner_id', 'winner_ht', 'winner_age', 'tourney_date']]
df_male_matches_reduced[df_male_matches_reduced['player_name'] == 'Aaron Addison']
#df_male_matches_reduced.groupby('player_name').agg('mean')

,player_name,winner_id,winner_ht,winner_age,tourney_date
31893,Aaron Addison,117357,-1,19,2016-10-24
31893,Aaron Addison,117360,-1,18,2016-02-01
31893,Aaron Addison,102864,193,18,2016-01-11
31893,Aaron Addison,105429,-1,26,2016-10-31
31893,Aaron Addison,111806,-1,23,2020-01-13
31893,Aaron Addison,105349,-1,30,2019-10-21
31893,Aaron Addison,106172,-1,25,2019-01-07
31893,Aaron Addison,200614,-1,21,2019-10-28
31893,Aaron Addison,200640,-1,19,2018-10-22


- name
- hand
- sex
- nationality
- win percentage
- tot match
- number of won tournment

In [108]:
df_player = pd.DataFrame()

list_of_male_names = df_male_players_tmp['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_male_names)}")
df_player['Name'] = list_of_male_names
df_player['Sex'] = 'M'
i = 0

for name in list_of_male_names:
    
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]

Lunghezza lista: 54556


df_players = pd.DataFrame()
df_players['name'] = np.append(df_male_players_tmp['player_name'].unique(), df_female_players_tmp['player_name'].unique())
df_players['won_matches'] = 0
df_players['lost_matches'] = 0

for index, row in df_matches.iterrows():
    print(index)
    df_players.loc[df_players.name == row['winner_name'], 'won_matches'] += 1
    df_players.loc[df_players.name == row['loser_name'], 'lost_matches'] += 1
    

In [110]:
df_player.to_csv("../data/cleaned/custom_male_players.csv", index=False)

In [111]:
df_player = pd.DataFrame()

list_of_female_names = df_female_players_tmp['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_female_names)}")
df_player['Name'] = list_of_female_names
df_player['Sex'] = 'F'
i = 0

for name in list_of_female_names:
    
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]

Lunghezza lista: 44089


In [113]:
df_player.to_csv("../data/cleaned/custom_female_players.csv", index=False)

In [ ]:
#### START FROM H

In [91]:
winners = df_matches[['winner_name', 'winner_id']]
losers = df_matches[['loser_name', 'loser_id']]

losers['lost'] = losers.groupby('loser_name')['loser_name'].transform('count')
winners['won'] = winners.groupby('winner_name')['winner_name'].transform('count')



/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,winner_name,winner_id,won
0,Kei Nishikori,105453,191
1,Daniil Medvedev,106421,267
2,Kei Nishikori,105453,191
3,Jo-Wilfried Tsonga,104542,125
4,Daniil Medvedev,106421,267
...,...,...,...
186084,Priscilla Hon,211880,110
186085,Ingrid Neel,214562,29
186086,Priscilla Hon,211880,110
186087,Veronika Kudermetova,206368,195


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de87b132-0371-4d99-94f9-c61923e2507d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>